In [70]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

from pathlib import Path
import pandas as pd
import numpy as np

fdir = Path.cwd()
print(fdir)
sys.path.append(str(fdir/'../src'))
from config import cfg

# from build_df import load_rna
# from merge_meta_files import load_crossref, load_pdx_meta
import load_data
from load_data import load_crossref, load_pdx_meta, load_pdx_meta2, load_pdx_meta_jc, load_rna, PDX_SAMPLE_COLS
from tfrecords import original_tfr_names

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/vol/ml/apartin/projects/pdx-histo/nbs


## Cref

In [2]:
cref = load_crossref(drop_bad_slides=False)
meta_slides = set(cref.image_id.values.tolist())
len(meta_slides)

593

In [3]:
svs_slides = set([s.name.split('.svs')[0] for s in cfg.SLIDES_DIR.glob('*.svs')])
len(svs_slides)

596

In [4]:
print("SVS slides that are present in the folder but not in the 'Image ID' " \
      "column: {}".format(svs_slides.difference(meta_slides)))
print("Slide ids that are in the 'Image ID' column but not present in the " \
      "folder:  {}".format(meta_slides.difference(svs_slides)))

SVS slides that are present in the folder but not in the 'Image ID' column: {'13504', '11008', '83741'}
Slide ids that are in the 'Image ID' column but not present in the folder:  set()


In [5]:
miss1 = svs_slides.difference(meta_slides)

In [6]:
tfrecords = original_tfr_names(label='299px_302um')
len(tfrecords)

A total of 590 original tfrecords.


590

In [7]:
set(tfrecords).intersection(miss1)

{'11008', '13504', '83741'}

### Judith

In [8]:
jj = load_pdx_meta_jc()
print(jj.shape)
jj[:2]

(1198, 8)


,Sample,patient_id,specimen_id,sample_id,csite_src,ctype_src,csite,ctype
0,114434~197-R~A35YC3,114434,197-R,A35YC3,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma
1,114434~197-R~A36YC9,114434,197-R,A36YC9,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma


### Yitan

In [9]:
yy = load_pdx_meta()
print(yy.shape)
yy[:2]

(97, 7)


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype,stage_or_grade
0,114434,197-R,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma,grade 3
1,114551,080-T,Head and Neck,Salivary gland cancer,head & neck,head & neck,grade 2


### Cross-ref

In [10]:
cref = load_crossref(drop_bad_slides=False)
print(cref.shape)
cref[:2]

(593, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


### Merge slides meta (cref) and PDX meta

In [9]:
print(cref.shape)
print(yy.shape)
display(cref[:2])
display(yy[:2])

(593, 5)
(97, 7)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype,stage_or_grade
0,114434,197-R,Musculoskeletal,Non-Rhabdo. soft tissue sarcoma,musculoskeletal,sarcoma/mesothelioma,grade 3
1,114551,080-T,Head and Neck,Salivary gland cancer,head & neck,head & neck,grade 2


In [10]:
# A few samples are missing
mrg_cols = ['patient_id', 'specimen_id']
m = cref.merge(yy[mrg_cols], on=mrg_cols, how='inner')
print(m.shape)
display(m[:2])

(584, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


In [11]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
mrg_outer = cref.merge(yy, on=mrg_cols, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss = miss.sort_values(mrg_cols, ascending=True)
print('\nMissing items', miss.shape)
display(miss)

Outer merge (593, 11)
both          584
left_only       9
right_only      0
Name: _merge, dtype: int64

Missing items (9, 11)


,model,patient_id,specimen_id,sample_id,image_id,csite_src,ctype_src,csite,ctype,stage_or_grade,_merge
45,146476~266-R,146476,266-R,E19YJ0,20252,NaN,NaN,NaN,NaN,NaN,left_only
46,146476~266-R,146476,266-R,E19YJ1,20307,NaN,NaN,NaN,NaN,NaN,left_only
47,146476~266-R,146476,266-R,E20F07,21545,NaN,NaN,NaN,NaN,NaN,left_only
48,146476~266-R,146476,266-R,E20F08F67,20976,NaN,NaN,NaN,NaN,NaN,left_only
49,146476~266-R,146476,266-R,ORIGINATOR,13946,NaN,NaN,NaN,NaN,NaN,left_only
148,237351~077-R,237351,077-R,AL-IR0,20729,NaN,NaN,NaN,NaN,NaN,left_only
149,237351~077-R,237351,077-R,AL-IR0_AL-M977,20467,NaN,NaN,NaN,NaN,NaN,left_only
150,237351~077-R,237351,077-R,AL-IR0_AL-M977_AL-M302,23429,NaN,NaN,NaN,NaN,NaN,left_only
151,237351~077-R,237351,077-R,AL-IR0_AL-M977_AL-M303_AL-ER0,26305,NaN,NaN,NaN,NaN,NaN,left_only


In [12]:
print(miss.patient_id.unique())
miss_found = jj[ jj.patient_id.isin(miss.patient_id.unique()) ]
miss_found = miss_found.drop(columns=['Sample', 'sample_id'])
miss_found = miss_found.drop_duplicates()
miss_found

['146476' '237351']


,patient_id,specimen_id,csite_src,ctype_src,csite,ctype
78,146476,266-R,Genitourinary,"Urothelial/bladder cancer, NOS",bladder/urothelial,bladder/urothelial
210,237351,077-R,Musculoskeletal,Ewing sarcoma/Peripheral PNET,musculoskeletal,sarcoma/mesothelioma


In [13]:
# Add the missing samples
yy_new = pd.concat([yy, miss_found], axis=0)
yy_new = yy_new.reset_index(drop=True)
print(yy_new.shape)

(99, 7)


In [14]:
# Now mege with the corrected file
mrg_cols = ['patient_id', 'specimen_id']
m = cref.merge(yy_new[mrg_cols], on=mrg_cols, how='inner')
print(m.shape)
display(m[:2])

(593, 5)


,model,patient_id,specimen_id,sample_id,image_id
0,114434~197-R,114434,197-R,A35YC3,27166
1,114434~197-R,114434,197-R,A36YC9,25127


In [15]:
# Save into file
yy_new.to_csv('../data/meta/PDX_Meta_Information2.csv', index=False)

# Merge for drug response
We want to find drug response samples (i.e. treatments) that are responders (Response=1) that don't have histology slides, BUT have RNA-Seq. <br>
There is a drug response pair with a response value of 1 (i.e. responder) for which we don't have a histology slide.

In [96]:
rsp = load_data.load_rsp()
rna = load_data.load_rna()
dd = load_data.load_dd()
cref = load_data.load_crossref(drop_bad_slides=False)
pdx = load_data.load_pdx_meta2()


Dropped 77 duplicates.

Descriptors with all NaN: 72
Descriptors with any NaN: 854
Drop descriptors with all NaN ...
Drop descriptors with a single unique value (excluding NaNs) ...
Impute NaN.
Descriptors with any NaN: 0


In [97]:
print(rsp.shape)
print(rna.shape)
rsp_rna = rsp.merge(rna, on='Sample', how='inner')
print(rsp_rna.shape)

(2591, 4)
(1727, 981)
(2591, 984)


In [98]:
print(rsp_rna.shape)
print(dd.shape)
rsp_rna_dd = rsp_rna.merge(dd, left_on='Drug1', right_on='ID', how='inner').reset_index(drop=True)
print(rsp_rna_dd.shape)

(2591, 984)
(2142, 1424)
(2591, 2408)


In [99]:
print(pdx.shape)
print(rsp_rna_dd.shape)
rsp_rna_dd_pdx = pdx.merge(rsp_rna_dd, on=['patient_id', 'specimen_id'], how='inner')
print(rsp_rna_dd_pdx.shape)

(99, 7)
(2591, 2408)
(2591, 2413)


In [100]:
print(cref.shape)
print(rsp_rna_dd_pdx.shape)
df = cref.merge(rsp_rna_dd_pdx, on=PDX_SAMPLE_COLS, how='inner')
print(df.shape)

(593, 5)
(2591, 2413)
(2591, 2414)


In [101]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
mrg_outer = cref.merge(rsp_rna_dd_pdx, on=PDX_SAMPLE_COLS, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss_r = mrg_outer.loc[lambda x: x['_merge']=='right_only']
miss_r = miss_r.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('\nMissing right items', miss_r.shape)

miss_l = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss_l = miss_l.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('Missing left items ', miss_l.shape)

Outer merge (2774, 2415)
both          2591
left_only      183
right_only       0
Name: _merge, dtype: int64

Missing right items (0, 2415)
Missing left items  (183, 2415)


In [102]:
m = df[df.image_id.isin(cfg.BAD_SLIDES)]
print(m.shape)

(17, 2414)


In [104]:
m

,model,patient_id,specimen_id,sample_id,image_id,csite_src,ctype_src,csite,ctype,stage_or_grade,...,dd_SRW10,dd_TSRW10,dd_MW,dd_AMW,dd_WPath,dd_WPol,dd_Zagreb1,dd_Zagreb2,dd_mZagreb1,dd_mZagreb2
788,269878~174-B,269878,174-B,PN3,22232,Skin,Merkel cell tumor,skin,skin other,nan,...,11.493304,102.403717,853.330933,7.551601,14767.0,129.0,346.0,429.0,23.361111,13.215278
789,269878~174-B,269878,174-B,PN3,22232,Skin,Merkel cell tumor,skin,skin other,nan,...,8.806724,37.839725,130.017853,10.834822,84.0,10.0,42.0,46.0,4.083333,2.027778
790,269878~174-B,269878,174-B,PN3,22232,Skin,Merkel cell tumor,skin,skin other,nan,...,10.007352,66.173042,263.071777,9.071440,560.0,31.0,96.0,116.0,7.979167,3.916667
791,269878~174-B,269878,174-B,PN3,22232,Skin,Merkel cell tumor,skin,skin other,nan,...,11.018613,95.764595,586.279114,7.238014,7049.0,86.0,246.0,306.0,12.840278,9.263889
792,269878~174-B,269878,174-B,PN3,22232,Skin,Merkel cell tumor,skin,skin other,nan,...,10.104344,46.164986,369.028839,16.044733,216.0,21.0,74.0,90.0,5.597222,2.666667
2076,773659~155-R,773659,155-R,NQ8K19M745,21836,Skin,Melanoma,skin,melanoma,nan,...,11.493304,102.403717,853.330933,7.551601,14767.0,129.0,346.0,429.0,23.361111,13.215278
2077,773659~155-R,773659,155-R,NQ8K19M745,21836,Skin,Melanoma,skin,melanoma,nan,...,8.806724,37.839725,130.017853,10.834822,84.0,10.0,42.0,46.0,4.083333,2.027778
2078,773659~155-R,773659,155-R,NQ8K19M745,21836,Skin,Melanoma,skin,melanoma,nan,...,10.007352,66.173042,263.071777,9.071440,560.0,31.0,96.0,116.0,7.979167,3.916667
2079,773659~155-R,773659,155-R,NQ8K19M745,21836,Skin,Melanoma,skin,melanoma,nan,...,10.104344,46.164986,369.028839,16.044733,216.0,21.0,74.0,90.0,5.597222,2.666667
2491,BL0382~F1232,BL0382,F1232,M662,83743,Genitourinary,"Urothelial/bladder cancer, NOS",bladder/urothelial,bladder/urothelial,grade 3,...,9.658929,60.153156,194.055222,9.702761,277.0,22.0,74.0,89.0,5.666667,3.055556


# Merge drug response and cref
We want to find drug response samples (i.e. treatments) that are responders (Response=1) that don't have histology slides. <br>
There is a drug response pair with a response value of 1 (i.e. responder) for which we don't have a histology slide.

In [82]:
rsp = load_data.load_rsp()
rsp = load_data.parse_Sample_col(rsp)
# rsp = rsp[rsp.Response == 1].reset_index(drop=True)

cref = load_data.load_crossref(drop_bad_slides=False)
pdx = load_data.load_pdx_meta2()


Dropped 77 duplicates.



In [83]:
# Explore (merge and identify from which df the items are coming from)
# https://kanoki.org/2019/07/04/pandas-difference-between-two-dataframes/
mrg_outer = cref.merge(rsp, on=PDX_SAMPLE_COLS, how='outer', indicator=True)
print('Outer merge', mrg_outer.shape)
print(mrg_outer['_merge'].value_counts())

miss_r = mrg_outer.loc[lambda x: x['_merge']=='right_only']
miss_r = miss_r.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('\nMissing right items', miss_r.shape)

miss_l = mrg_outer.loc[lambda x: x['_merge']=='left_only']
miss_l = miss_l.sort_values(PDX_SAMPLE_COLS, ascending=True)
print('Missing left items ', miss_l.shape)

Outer merge (2774, 10)
both          2591
left_only      183
right_only       0
Name: _merge, dtype: int64

Missing right items (0, 10)
Missing left items  (183, 10)


In [84]:
print(cref.shape)
print(rsp.shape)
df = cref.merge(rsp, on=PDX_SAMPLE_COLS, how='inner', indicator=True)
print(df.shape)

(593, 5)
(2591, 8)
(2591, 10)


In [85]:
m = df[df.image_id.isin(cfg.BAD_SLIDES)]
print(m.shape)

(17, 10)


In [86]:
print(sorted(m.image_id.unique()))

['21836', '22232', '83742', '83743']


In [88]:
m[m.Response==1]

,model,patient_id,specimen_id,sample_id,image_id,smp,Sample,Drug1,Response,_merge
788,269878~174-B,269878,174-B,PN3,22232,269878~174-B~PN3_NSC.125973,269878~174-B~PN3,NSC.125973,1,both
